In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark Example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
accounts = spark.read.option('header',True).csv('/FileStore/tables/accounts-2.csv')
accounts

Out[4]: DataFrame[account_number: string, aba: string, bic: string, opened: string, balance: string]

In [0]:
accounts.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- aba: string (nullable = true)
 |-- bic: string (nullable = true)
 |-- opened: string (nullable = true)
 |-- balance: string (nullable = true)



In [0]:
transactions = spark.read.option('header', True).parquet('/FileStore/tables/transactions.parquet')

In [0]:
transactions.count()

Out[7]: 1000000

In [0]:
account_transactions = transactions.groupby('account_number').sum()
with_sum = accounts.join(account_transactions, 'account_number', 'inner')
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))

In [0]:
accounts.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- aba: string (nullable = true)
 |-- bic: string (nullable = true)
 |-- opened: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- sum(amount): long (nullable = true)
 |-- new_balance: double (nullable = true)



In [0]:
neg_balance = accounts.filter(accounts.new_balance < 0)

In [0]:
clients = spark.read.json('/FileStore/tables/clients.json')

In [0]:
clients.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- address: string (nullable = true)
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)



In [0]:
clients = clients.join(neg_balance, 'account_number', 'inner')

In [0]:
clients.select(['first_name','last_name', 'account_number','new_balance']).show(5)

+----------+---------+------------------+-----------+
|first_name|last_name|    account_number|new_balance|
+----------+---------+------------------+-----------+
|    Meagan| Sandoval|JMTP45763117901514|   -27573.0|
|  Michelle|   Knight|RBUE05237750254383|  -103459.0|
|      Paul|   Massey|RJMY57096756148587|   -58329.0|
|  Michelle|    Perez|ZYMB62177146259441|   -55431.0|
|     David|    Green|LRTH65732611614073|  -103831.0|
+----------+---------+------------------+-----------+
only showing top 5 rows

